In [1]:
# 카테고리아이디 1로 초기화
import os
import json
from tqdm import tqdm

# 경로 설정
directory = r'C:\Users\Administrator\Downloads\datasets\labels'

# 파일 수 세기
num_files = len([f for f in os.listdir(directory) if f.endswith('.json')])

# 경로의 모든 파일 순회
for filename in tqdm(os.listdir(directory), total=num_files, unit="file"):
    if filename.endswith('.json'):
        # JSON 파일 경로 구성
        file_path = os.path.join(directory, filename)
        
        # JSON 파일 열기 및 데이터 로드
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # "category_id" 필드의 값을 0로 변경
        for annotation in data['annotations']:
            annotation['category_id'] = 0
        
        # 변경된 데이터를 파일에 저장
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

print('작업이 완료되었습니다.')

100%|██████████████| 3591/3591 [01:00<00:00, 59.74file/s]

작업이 완료되었습니다.


In [ ]:
# import glob

# # 파일 경로 지정
# file_path = '/content/drive/MyDrive/yolov5-master/DarkInsight/data/labels/val/*.txt'

# # 각 파일에 대해 처리
# for file in glob.glob(file_path):
#     with open(file, 'r') as f:
#         lines = f.readlines()

#     # 각 라인의 첫번째 숫자를 0으로 변경
#     modified_lines = [line.replace('1 ', '0 ') for line in lines]

#     # 변경된 내용을 파일에 다시 쓰기
#     with open(file, 'w') as f:
#         f.writelines(modified_lines)

# print('파일 처리가 완료되었습니다.')

In [3]:
import os
import json
import random
import shutil
from sklearn.model_selection import train_test_split
# 디렉터리 경로 설정
dataset_dir = r'C:\Users\Administrator\Downloads\datasets'
img_dir = os.path.join(dataset_dir, 'images')
label_dir = os.path.join(dataset_dir, 'labels_json')
# 이미지와 라벨 파일 목록 가져오기
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.jpg')])
json_files = sorted([f for f in os.listdir(label_dir) if f.endswith('.json')])
# 데이터셋을 train, val, test로 분할
train_imgs, test_imgs, train_jsons, test_jsons = train_test_split(img_files, json_files, test_size=0.2, random_state=42)
train_imgs, val_imgs, train_jsons, val_jsons = train_test_split(train_imgs, train_jsons, test_size=0.25, random_state=42)
# 분할된 데이터셋 디렉터리 생성
os.makedirs(os.path.join(dataset_dir, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'train', 'labels_json'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'val', 'labels_json'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'test', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'test', 'labels_json'), exist_ok=True)
# 파일 이동 함수
def move_files(files, src_dir, dest_dir):
    for file in tqdm(files):
        shutil.move(os.path.join(src_dir, file), os.path.join(dest_dir, file))
# 파일 이동
move_files(train_imgs, img_dir, os.path.join(dataset_dir, 'train', 'images'))
move_files(train_jsons, label_dir, os.path.join(dataset_dir, 'train', 'labels_json'))
move_files(val_imgs, img_dir, os.path.join(dataset_dir, 'val', 'images'))
move_files(val_jsons, label_dir, os.path.join(dataset_dir, 'val', 'labels_json'))
move_files(test_imgs, img_dir, os.path.join(dataset_dir, 'test', 'images'))
move_files(test_jsons, label_dir, os.path.join(dataset_dir, 'test', 'labels_json'))

100%|████████████████| 719/719 [00:00<00:00, 1452.17it/s]


In [20]:
# 트레인 테스트 벨리데이션으로 나기기
import os
import json
import numpy as np

# 동시에 각각 한줄 씩 주석 해제 하고 실행

# JSON 라벨 파일 경로
# json_label_dir = r'C:\Users\Administrator\Downloads\datasets\test\labels_json'
# json_label_dir = r'C:\Users\Administrator\Downloads\datasets\train\labels_json'
json_label_dir = r'C:\Users\Administrator\Downloads\datasets\val\labels_json'

# YOLO 라벨 파일 저장 경로
# yolo_label_dir = r'C:\Users\Administrator\Downloads\datasets\test\labels'
# yolo_label_dir = r'C:\Users\Administrator\Downloads\datasets\train\labels'
yolo_label_dir = r'C:\Users\Administrator\Downloads\datasets\val\labels'

# JSON 파일 읽기
for filename in os.listdir(json_label_dir):
    if filename.endswith('.json'):
        with open(os.path.join(json_label_dir, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 이미지 크기 가져오기
        img_width = data['images'][0]['width']
        img_height = data['images'][0]['height']

        # YOLO 형식으로 변환
        yolo_labels = []
        for obj in data['annotations']:
            class_id = obj['category_id']
            x, y, w, h = obj['bbox']
            x_center = x + w / 2
            y_center = y + h / 2
            width = w
            height = h
            yolo_labels.append(f"{class_id} {x_center / img_width} {y_center / img_height} {width / img_width} {height / img_height}")

        # YOLO 라벨 파일 저장
        img_filename = os.path.splitext(filename)[0] + '.jpg'
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(yolo_label_dir, label_filename)
        with open(label_path, 'w') as f:
            f.write('\n'.join(yolo_labels))


# yolov5/data 디렉터리 images, labels 디렉터리 각각 만들기
# images 디렉터리 하위에 train, test, val 디렉터리 생성후 각각 디렉터리에 이미지 업로드
# labels 디렉터리 하위에 train, test, val 디렉터리 생성 후 각각 디렉터리에 이미지에 상응하는 txt 데이타 업로드

<!-- set GIT_PYTHON_GIT_EXECUTABLE=C:\Users\Administrator\AppData\Local\GitHubDesktop\app-3.3.18\resources\app\git\cmd\git.exe -->
<!-- python train.py --data data/custom.yaml --weights yolov5s.pt --cfg yolov5s.yaml --epochs 100 --batch-size 16 --patience 5 -->